## Setup

In [20]:
from betterMT5 import connected
from backtesting import backtesting
from datetime import datetime
import cufflinks as cf
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
with connected():
    test = backtesting.Backtest("./chats/results_daniel.json", verbose=None)

In [3]:
with connected():
    results = test.run()

In [28]:
results = pd.DataFrame(test.make_results(test.run_results))
results.drop(results[results.result>4].index, axis=0, inplace=True)
results.drop(results[results.sl_pips>50].index, axis=0, inplace=True)
results.drop(results[results.sl_pips<5].index, axis=0, inplace=True)
results.open = pd.to_datetime(results.open.apply(str), utc=True)
results.close = pd.to_datetime(results.close.apply(str), utc=True)
results['cum'] = results['result'].cumsum()
results.to_csv(f'results/test_run_{datetime.utcnow().strftime("%Y%m%dT%H%M")}.csv')
results

,open,close,symbol,sl_pips,result,type,cum
0,2020-11-05 12:46:00+00:00,2020-11-05 13:05:00+00:00,GBPJPY,25.9,-1.000000,SL,-1.000000
1,2020-11-05 13:26:28+00:00,2020-11-05 13:42:00+00:00,GBPJPY,13.7,-1.000000,SL,-2.000000
2,2020-11-06 02:08:20+00:00,2020-11-06 02:31:00+00:00,GBPJPY,12.2,-0.032787,CLOSE,-2.032787
3,2020-11-09 09:32:23+00:00,2020-11-09 09:57:00+00:00,GBPJPY,24.8,-0.036290,SL_TO_BE,-2.069077
4,2020-11-10 23:38:12+00:00,2020-11-11 00:06:00+00:00,GBPJPY,10.3,-1.000000,SL,-3.069077
...,...,...,...,...,...,...,...
414,2022-01-25 08:18:06+00:00,2022-01-25 09:22:00+00:00,GBPJPY,22.6,0.991150,TP,9.700228
415,2022-01-26 10:14:32+00:00,2022-01-26 11:20:00+00:00,GBPJPY,21.3,0.877934,TP,10.578162
416,2022-01-27 09:44:42+00:00,2022-01-27 10:37:00+00:00,GBPJPY,19.7,0.984772,TP,11.562934
417,2022-01-31 16:06:49+00:00,2022-01-31 16:42:00+00:00,GBPJPY,24.2,0.657025,TP,12.219959


## Results (Vanilla Daniel Savage)

In [29]:
data_loss = len(test.trades) - results.shape[0]
print(f'Post-adjustment data loss: {data_loss} ({data_loss/len(test.trades):.2%})')

Post-adjustment data loss: 21 (4.90%)


In [30]:
results.iplot(kind='scatter', mode='markers', x='close', title='Results (R) by type', yTitle='R',
              y='result', categories='type', size=6, colors='red blue purple green grey black'.split(' '))

In [58]:
results['sl_pips'].iplot(kind='hist', title='SL (pips) distribution', yTitle='pips', xTitle='index', opacity=1, color='#68D8D6', linecolor='#1985F0')

In [59]:
results['duration_minutes'] = pd.to_timedelta(results.close - results.open).apply(lambda x: x.total_seconds()/60)
results['duration_minutes'].iplot(kind='hist', title='Trades length distribution (minutes)',
    yTitle='minutes', xTitle='index', color='#68D8D6', opacity=1, linecolor='#1985F0')

In [63]:
interpolated = results.set_index('close').resample('3600T').mean().interpolate(method='spline', order=3)
interpolated.iplot(kind='scatter', y='cum', bestfit=True, color='#0C120C', fill=True)
del interpolated

In [70]:
res_gb_M = results.set_index('close').groupby(pd.Grouper(freq='M')).sum()[['result']]
res_gb_M['result_pos'] = res_gb_M.result[res_gb_M.result>=0]
res_gb_M['result_neg'] = res_gb_M.result[res_gb_M.result<0]
res_gb_M.drop('result', inplace=True, axis=1)
res_gb_M

,result_pos,result_neg
close,,
2020-11-30 00:00:00+00:00,NaN,-2.973839
2020-12-31 00:00:00+00:00,NaN,-1.111851
2021-01-31 00:00:00+00:00,1.130078,NaN
2021-02-28 00:00:00+00:00,NaN,-1.119601
2021-03-31 00:00:00+00:00,3.874042,NaN
2021-04-30 00:00:00+00:00,NaN,-0.163038
2021-05-31 00:00:00+00:00,3.559188,NaN
2021-06-30 00:00:00+00:00,NaN,-1.596187
2021-07-31 00:00:00+00:00,1.800060,NaN


In [71]:
avg = res_gb_M.result_pos.mean() + res_gb_M.result_neg.mean()
res_gb_M.iplot(kind='bar', bargap=0, barmode='stack', colors=dict(result_pos='seagreen', result_neg='#FF0000'), hline=avg)

In [77]:

res_gb_D = results.set_index('open')[['result']].groupby(pd.Grouper(freq='D')).sum()
res_gb_D['result_pos'] = res_gb_D.result[res_gb_D.result>=0]
res_gb_D['result_neg'] = res_gb_D.result[res_gb_D.result<0]
res_gb_D.drop('result', inplace=True, axis=1)
res_gb_D

,result_pos,result_neg
open,,
2020-11-05 00:00:00+00:00,NaN,-2.000000
2020-11-06 00:00:00+00:00,NaN,-0.032787
2020-11-07 00:00:00+00:00,0.000000,NaN
2020-11-08 00:00:00+00:00,0.000000,NaN
2020-11-09 00:00:00+00:00,NaN,-0.036290
...,...,...
2022-01-28 00:00:00+00:00,0.000000,NaN
2022-01-29 00:00:00+00:00,0.000000,NaN
2022-01-30 00:00:00+00:00,0.000000,NaN


In [78]:
avg = res_gb_D.result_pos.mean() + res_gb_D.result_neg.mean()
res_gb_D.iplot(kind='scatter', mode='markers', colors=dict(result_pos='seagreen', result_neg='#FF0000'), hline=avg)

In [80]:
results.set_index('open').loc['2021-12-7']

,close,symbol,sl_pips,result,type,cum,duration_minutes
open,,,,,,,
2021-12-07 08:12:08+00:00,2021-12-07 08:33:00+00:00,GBPJPY,17.9,-1.0,SL,1.104287,20.866667
2021-12-07 08:52:00+00:00,2021-12-07 09:09:00+00:00,GBPJPY,19.8,-1.0,SL,0.104287,17.000000
2021-12-07 09:17:55+00:00,2021-12-07 10:52:00+00:00,GBPJPY,20.7,-1.0,SL,-0.895713,94.083333
2021-12-07 11:01:07+00:00,2021-12-07 11:23:00+00:00,GBPJPY,13.9,-1.0,SL,-1.895713,21.883333
2021-12-07 12:23:18+00:00,2021-12-07 13:18:00+00:00,GBPJPY,23.0,-1.0,SL,-2.895713,54.700000
